In [1]:
import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.test.utils import datapath
# spacy for lemmatization
import ru2
import spacy
# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\mdere\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
#!pip install spacy==2.1.9
#!pip install pymorphy2==0.8
#!pip uninstall msgpack-python

In [6]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mdere\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
df = pd.read_csv('dataset.csv', index_col=0)
data = df['Review Text'].tolist()

In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['не', 'скачивается', 'стала', 'обновлять', 'зависло', 'приложение', 'что', 'случилось']]


In [13]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

C:\Users\mdere\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['не', 'скачивается', 'стала', 'обновлять', 'зависло', 'приложение', 'что', 'случилось']


In [14]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('ru2')
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])


[['скачиваться', 'стать', 'обновлять', 'зависнуть', 'приложение', 'случиться']]


In [16]:
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [21]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=120, 
                                           random_state=100,
                                           update_every=10,
                                           chunksize=10000,
                                           passes=10,
                                           per_word_topics=True)

C:\Users\mdere\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [24]:
= datapath('C:\\Users\\mdere\\source\\repos\\SberOnline\\Code\\model2\\model')

In [25]:
lda_model.save(temp_file)

C:\Users\mdere\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [17]:
lda_model = gensim.models.ldamodel.LdaModel.load(temp_file)

In [22]:
lda_model.show_topics(num_topics=120)

[(0,
  '0.046*"приложение" + 0.038*"самом" + 0.020*"бог" + 0.019*"банк" + 0.017*"удобный" + 0.016*"пин" + 0.014*"очень" + 0.014*"диван" + 0.012*"основнои" + 0.010*"крупный"'),
 (1,
  '0.040*"приложение" + 0.025*"ехать" + 0.016*"очень" + 0.016*"муж" + 0.015*"старую_версию" + 0.013*"удачный" + 0.012*"телефон" + 0.012*"сбербанк" + 0.011*"связать" + 0.010*"удобный"'),
 (2,
  '0.027*"период" + 0.022*"списание" + 0.020*"обновляться" + 0.017*"контроль" + 0.017*"магазин" + 0.016*"остаток" + 0.015*"видеть" + 0.015*"приложение" + 0.014*"карта" + 0.014*"трата"'),
 (3,
  '0.255*"спасибо" + 0.116*"очень" + 0.097*"удобный" + 0.092*"большой" + 0.065*"приложение" + 0.031*"лёгкий" + 0.028*"огромный" + 0.023*"удобство" + 0.015*"сбербанк" + 0.014*"работа"'),
 (4,
  '0.064*"получить" + 0.021*"уходить" + 0.021*"сутки" + 0.021*"списывать" + 0.020*"сбербанке" + 0.019*"деньга" + 0.019*"банк" + 0.015*"справка" + 0.015*"документ" + 0.013*"сделать"'),
 (5,
  '0.046*"приложение" + 0.045*"скачивать" + 0.044*"специ

In [23]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.97867563431096

Coherence Score:  0.39028887828455816


In [20]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

KeyboardInterrupt: 

In [ ]:
def get_with_threshold(inputs, threshold):
    result = []
    for index, elem in enumerate(inputs):
        if elem >= threshold:
            result.append(index + 1)
    return result

In [46]:
df = pd.read_csv('answer_dataset.csv', index_col=0)
df.head()

,Developer Reply Text,Review Text
56,"Рады, что вам понравилось приложение!\nПоясним...",Разработчики молодцы Приложение достойное Но в...
57,У нас никаких сбоев не было зафиксировано. Поз...,Почему я не могу сегодня целый день зайти в пр...
59,"В нашем приложении нет рекламы, есть лишь пред...",Забили рекламой хорошее приложение Отключить н...
60,"Исходя из статистики, большинство наших клиент...",В условиях пандемии ввести комиссию говорит о ...
66,Включить возможность скачивания приложений бол...,Невозможно скачать приложение с новым обновлен...


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45639 entries, 56 to 130102
Data columns (total 2 columns):
Developer Reply Text    45639 non-null object
Review Text             45639 non-null object
dtypes: object(2)
memory usage: 1.0+ MB


In [38]:
def distance(a, b):
    n, m = len(a), len(b)
    if n > m:
        a, b = b, a
        n, m = m, n

    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if a[j - 1] != b[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

In [85]:
import sys

def get_answer(text, df):
    result = ""
    dist = sys.maxsize
    for entity in df.values:
        new_dist = distance(text[:25], entity[1][:25])
        if new_dist <= dist:
            dist = new_dist
            result = entity[0]
    return result

In [88]:
get_answer("Как мне завершить платеж если нет интернета", df[:2000])

'При зависании рекомендуем выгрузить приложение из памяти телефона, перезагрузить устройство и войти в приложение снова. Добавим, что медленная работа программы, как правило, связана с нестабильным интернетом, попробуйте войти, используя другое соединение.'